In [93]:
!pip install -U langchain-ollama langchain-openai

In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
!ls /content/drive/MyDrive/review_dataset

w_review_train.csv  w_review_train.parquet


In [96]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [97]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [98]:
df.shape

(40000, 2)

In [99]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [100]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [101]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [102]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [103]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [104]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.37 s, sys: 26.9 ms, total: 2.39 s
Wall time: 4.89 s


In [105]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [106]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 5.1 s, sys: 55.5 ms, total: 5.15 s
Wall time: 9.42 s


In [107]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [108]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [92]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [109]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [110]:
!ollama pull gemma3:270m

In [111]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
gemma3:270m                           e7d36fb2c3b3    291 MB    Less than a second ago    
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    6 minutes ago             


In [112]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ ยินดีที่ได้รู้จักค่ะ 😊




In [113]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-19T07:54:56.858565281Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}},{"name":"scb10x/typhoon2.1-gemma3-4b:latest","model":"scb10x/typhoon2.1-gemma3-4b:latest","modified_at":"2025-10-19T07:48:43.083361103Z","size":2615662352,"digest":"8cfab9097c9d142fa16c7556826c9a7874f4642ac6d575e2d66a4f4979c48b76","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"3.9B","quantization_level":"Q4_K_M"}}]}

In [114]:
!pip install -U langchain-ollama langchain-openai
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [115]:
llm = OllamaLLM(model="gemma3:270m")

In [116]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ ยินดีที่ได้รู้จักค่ะ 😊



In [117]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [118]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [119]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [120]:
chain = prompt | llm

In [121]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [122]:
response

'```python\ndef find_sum_of_numbers(numbers):\n  """\n  หาผลรวมของตัวเลข 1 ถึง 10\n  """\n  total = 0\n  for number in numbers:\n    total += number\n  return total\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nresult = find_sum_of_numbers(numbers)\nprint("ผลรวม:", result)\n```\n\n**คำอธิบาย:**\n\n1.  **`def find_sum_of_numbers(numbers):`**:  ชื่อของฟังก์ชันคือ `find_sum_of_numbers` ซึ่งเป็นชื่อที่น่าสนใจและสื่อถึงหน้าที่หลักของมันคือการหาผลรวมของตัวเลข 1 ถึง 10\n\n2.  **`total = 0`**:  ตัวแปร `total` จะถูกใส่เป็นค่าเริ่มต้นเพื่อให้สามารถคำนวณผลรวมได้\n\n3.  **`for number in numbers:`**:  ฟังก์ชันนี้จะวนซ้ำผ่านแต่ละตัวเลขใน `numbers`  (ซึ่งเป็นตัวแปรของ `numbers`)  เราจะทำการคำนวณตัวเลขต่อไป\n\n4.  **`total += number`**:  ในแต่ละรอบ เราจะทำการคำนวณตัวเลข `number`  และเพิ่มค่าที่ได้เข้าไปใน `total`  (ซึ่งเป็นตัวแปรของ `total`)\n\n5.  **`return total`**:  หลังจากคำนวณทั้งหมดแล้ว `total` จะเป็นค่าที่คำนวณได้สำเร็จ  เราจะคืนค่าค่านี้\n\n**ตัวอย่างการใช้งาน:**\n\n```pyt

In [123]:
from IPython.display import display, Markdown

In [124]:
display(Markdown(response))

```python
def find_sum_of_numbers(numbers):
  """
  หาผลรวมของตัวเลข 1 ถึง 10
  """
  total = 0
  for number in numbers:
    total += number
  return total

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum_of_numbers(numbers)
print("ผลรวม:", result)
```

**คำอธิบาย:**

1.  **`def find_sum_of_numbers(numbers):`**:  ชื่อของฟังก์ชันคือ `find_sum_of_numbers` ซึ่งเป็นชื่อที่น่าสนใจและสื่อถึงหน้าที่หลักของมันคือการหาผลรวมของตัวเลข 1 ถึง 10

2.  **`total = 0`**:  ตัวแปร `total` จะถูกใส่เป็นค่าเริ่มต้นเพื่อให้สามารถคำนวณผลรวมได้

3.  **`for number in numbers:`**:  ฟังก์ชันนี้จะวนซ้ำผ่านแต่ละตัวเลขใน `numbers`  (ซึ่งเป็นตัวแปรของ `numbers`)  เราจะทำการคำนวณตัวเลขต่อไป

4.  **`total += number`**:  ในแต่ละรอบ เราจะทำการคำนวณตัวเลข `number`  และเพิ่มค่าที่ได้เข้าไปใน `total`  (ซึ่งเป็นตัวแปรของ `total`)

5.  **`return total`**:  หลังจากคำนวณทั้งหมดแล้ว `total` จะเป็นค่าที่คำนวณได้สำเร็จ  เราจะคืนค่าค่านี้

**ตัวอย่างการใช้งาน:**

```python
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum_of_numbers(numbers)
print("ผลรวม:", result)
```

**การทำงานของโค้ด:**

*   `find_sum_of_numbers(numbers)`  ทำงานโดยการลบตัวแปร `numbers`  (ซึ่งเป็นตัวแปรของ `numbers`)  ออกจาก `total`  (ซึ่งเป็นตัวแปรของ `total`)  และทำการคำนวณผลรวม
*   `return total`  จะคืนค่าค่า `total`  ซึ่งเป็นผลรวมของตัวเลข 1 ถึง 10

**ข้อดีของโค้ดนี้:**

*   **ง่ายต่อการเข้าใจ:**  โค้ดนี้เป็นภาษา Python ที่เข้าใจง่าย และสามารถนำไปใช้ได้กับหลายภาษา
*   **ใช้ภาษา Python:**  โค้ดนี้ใช้ภาษา Python ซึ่งเป็นภาษาที่ได้รับความนิยมอย่างมากสำหรับการเขียนโปรแกรม
*   **มีประสิทธิภาพ:**  โค้ดนี้ทำงานได้อย่างรวดเร็ว
*   **สามารถปรับแต่งได้:**  โค้ดนี้สามารถปรับแต่งได้โดยการเพิ่มหรือลดตัวแปรใน `numbers`  เพื่อให้ได้ผลลัพธ์ที่เหมาะสมกับข้อมูลที่คุณต้องการ



In [125]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [126]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [127]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [128]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [129]:
parser_chain = StrOutputParser()

In [130]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [131]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [132]:
result

'ถูกต้องแล้วครับ'

In [133]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [134]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [135]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [136]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [137]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 52.1 ms, sys: 5.16 ms, total: 57.3 ms
Wall time: 8.75 s


In [138]:
for result in results:
    print(f"Review {result['review'][:1000]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย 
ลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่
ทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล 
พะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว
เลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) 

คงไม่ไปซ้ำ แพงเกิน ...
text ```json
{"keywords": ["ร้านอาหาร", "ห้องน้ำ", "กาแฟ", "น้ำผึ้ง", "น้ำผึ้ง", "ลาบไข่ต้ม", "ไข่มันคาว", "ขนมไทย", "ขนมหวาน", "ฟรี", "ราคา", "เกินไป", "รับไม่ไหว", "ว", "ไม่ไปซ้ำ", "แพงเกิน"}
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก
เรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป
present งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี
จัดไป.....


บรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี
อย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ
เขียวช

In [139]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [140]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    Less than a second ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    45 seconds ago            


In [141]:
llm3 = OllamaLLM(model="scb10x/typhoon2.1-gemma3-4b")

In [142]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 1.72 s, sys: 136 ms, total: 1.86 s
Wall time: 1min 18s


In [143]:
for result in results:
    print(f"Review {result['review'][:1000]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย 
ลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่
ทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล 
พะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว
เลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) 

คงไม่ไปซ้ำ แพงเกิน ...
text ```json
{"keywords": ["ร้านอาหาร", "ห้องน้ำ", "กาแฟ", "น้ำผึ้ง", "น้ำผึ้ง", "ลาบไข่ต้ม", "ไข่มัน", "ทอดมัน", "หัวปลีก", "อร่อย", "เบิ้ล", "ขนมไทย", "ขนมหวาน", "ฟรี", "ขนมปลอมจิสติก", "ขนมไทย", "ขนมไข่ต้ม", "ขนมข้าว"]}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก
เรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป
present งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี
จัดไป.....


บรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี
อย่างน้อยก็เป็นสีเขียวเล็ก

In [144]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [145]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [146]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [147]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 153 ms, sys: 7.52 ms, total: 161 ms
Wall time: 16.7 s


In [148]:
results[0]['text']['keywords']

['ราคา', 'รสชาติ', 'ขนาด']

In [149]:
results[1]['text']['keywords']

['วาวี', 'ลาเต้', 'บรรยากาศ']

In [150]:

llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [151]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 148 ms, sys: 9.1 ms, total: 157 ms
Wall time: 13.6 s


In [152]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'อาหาร', 'ขนาด'], ['กาแฟ', 'บรรยากาศ', 'รสชาติ'])

In [153]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [154]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [155]:
inputs = [{"review": r} for r in sample_reviews]

In [156]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 22.8 ms, sys: 2.64 ms, total: 25.4 ms
Wall time: 2.55 s


In [157]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [158]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(1000).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 8

results_1000 = []

In [159]:
from tqdm import tqdm

In [160]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 125/125 [20:39<00:00,  9.92s/it]


In [162]:
for result in results_1000[0:10]:
    print(result['text']['keywords'])

ราคา, บรรยากาศ, ไข่ต้ม
กาแฟ, บรรยากาศ, รสชาติ
โดนัท, รสชาติ, ความหวาน
วาซาบิ, ปลาซาบะ, ข้าวหน้าปลา
โดนัท, Original Glazed, คิว
วังพญาไท, นรสิงห์, คาเฟ่
อาหารทะเล, ราคา, บรรยากาศ
กาแฟ, เค้ก, พนักงาน
ฮันนี่ โทสท์, ราคา, รสชาติ
คาเฟ่, วานิลลา, คาโบนาร่า


In [163]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [164]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)

In [165]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [166]:
context_extract = context_prompt | llm3 | context_parser

In [167]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [168]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [169]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords'].split(',')
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 1000/1000 [00:00<00:00, 240941.18it/s]


In [170]:
len(context_inputs)

3000

In [171]:
context_results = []
for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 1000/1000 [1:16:32<00:00,  4.59s/it]


In [172]:
results_1000[:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'บรรยากาศร้านใหญ่มากกกกกกก',
   'ไข่ต้ม']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ\nเขียวชอุ่ม\n\nว่าแล้วไม่รอช้

In [174]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 1000/1000 [00:00<00:00, 182258.03it/s]


In [175]:
len(sentiment_inputs)

3000

In [176]:
sentiment_inputs[:6]

[{'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'บรรยากาศร้านใหญ่มากกกกกกก'},
 {'message': 'ไข่ต้ม'},
 {'message': 'กาแฟที่นี่อร่อยมากกกกกกก เรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง'},
 {'message': 'บรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี อย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ เขียวชอุ่ม'},
 {'message': 'รสชาติและกลิ่นมันไม่ธรรมดาเลย หอมๆ ถือว่าเป็นมวยรุ่นเดียวกะ Starbuck ได้'}]

In [177]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 1000/1000 [44:46<00:00,  2.69s/it]


In [178]:
results_1000[0:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'บรรยากาศร้านใหญ่มากกกกกกก',
   'ไข่ต้ม'],
  'sentiment': ['negative', 'positive', 'neutral']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียวเล

In [179]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])


In [180]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')